# Library / Packages

In [1]:
# basic library
import os
import pandas as pd
import numpy as np
import sys

# pickle and .env
from dotenv import dotenv_values
import pickle

# Dataset

In [2]:
# 
pd.set_option('display.max_columns', None)

## Credit Info

In [3]:
# parameter
link = {**dotenv_values('../.env.secret')}
path = link['RAW_CREDIT_INFO']

# filter missing values
missing_val = ['N/a', 'n/a', 'No', 'N\a', 'na', 'NA', np.nan]

# load data to df
cc_info_df = pd.read_csv(path, sep = ',', na_values = missing_val, low_memory = False)
cc_info_df.head()

,credit_card,city,state,zipcode,credit_card_limit
0,1280981422329509,Dallas,PA,18612,6000
1,9737219864179988,Houston,PA,15342,16000
2,4749889059323202,Auburn,MA,1501,14000
3,9591503562024072,Orlando,WV,26412,18000
4,2095640259001271,New York,NY,10001,20000


In [4]:
# check data type
cc_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   credit_card        984 non-null    int64 
 1   city               984 non-null    object
 2   state              984 non-null    object
 3   zipcode            984 non-null    int64 
 4   credit_card_limit  984 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 38.6+ KB


### Data Cleaning

#### Category Check

In [5]:
check_cat = cc_info_df.select_dtypes(include = ['object'])
check_cat.head()

,city,state
0,Dallas,PA
1,Houston,PA
2,Auburn,MA
3,Orlando,WV
4,New York,NY


In [6]:
for i in check_cat.columns:
    print(f'{i.upper()} \t: {check_cat[i].unique()} \n')
    print(f'{check_cat[i].value_counts()} \n')
    print(f'{'-' * 50} \n')

CITY 	: ['Dallas' 'Houston' 'Auburn' 'Orlando' 'New York' 'Atlanta' 'Pittsburgh'
 'Birmingham' 'Washington' 'Los Angeles' 'Phoenix' 'Colorado Springs'
 'San Francisco' 'Columbus' 'Topeka' 'El Paso' 'Corpus Christi' 'Richmond'
 'Sacramento' 'Arlington' 'Cleveland' 'Tacoma' 'San Diego' 'Springfield'
 'Chicago' 'Seattle' 'Albany' 'Kansas City' 'Denver' 'Oklahoma City'
 'Las Vegas' 'Raleigh' 'Pasadena' 'Memphis' 'Des Moines' 'Miami'
 'Portland' 'Buffalo' 'Long Beach' 'Newark' 'San Antonio' 'Indianapolis'
 'Fort Worth' 'Akron' 'New Orleans' 'Saint Louis' 'Salt Lake City'
 'Fresno' 'Cincinnati' 'Austin' 'Dover' 'San Jose' 'Trenton' 'Oakland'
 'Gretna' 'Clinton' 'Honolulu' 'Charleston' 'Dayton' 'Mobile' 'Charlotte'
 'Philadelphia' 'Madison' 'Newport' 'Lafayette' 'Pensacola' 'Jacksonville'
 'Greensboro' 'Shreveport' 'Saint Paul' 'Huntington' 'Jackson' 'Lexington'
 'Rochester' 'Minneapolis' 'Columbia' 'Louisville' 'Boston' 'New Haven'
 'Wichita' 'Bristol' 'Hillsboro' 'Hartford' 'Friendship' 'Om

#### Data Duplicate

In [7]:
# check general duplicate
print(f"Total General Duplicate: {cc_info_df.duplicated().sum()}")

Total General Duplicate: 0


In [8]:
# check specific duplicate
print(f"Total Specific Duplicate: {cc_info_df['credit_card'].duplicated().sum()}")

Total Specific Duplicate: 0


#### Null Checking

In [9]:
# check null
cc_info_df.isnull().sum()

credit_card          0
city                 0
state                0
zipcode              0
credit_card_limit    0
dtype: int64

## Credit Transactions

In [10]:
# parameter
link = {**dotenv_values('../.env.secret')}
path = link['RAW_CREDIT_TRANSACTIONS']

# filter missing values
missing_val = ['N/a', 'n/a', 'No', 'N\a', 'na', 'NA', np.nan]

# load data to df
cc_trans_df = pd.read_csv(path, sep = ',', na_values = missing_val, low_memory = False)
cc_trans_df.head()

,credit_card,date,transaction_dollar_amount,Long,Lat
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719


In [11]:
# 
cc_trans_df.columns = cc_trans_df.columns.str.lower().str.replace(' ', '_')
cc_trans_df = cc_trans_df.rename(columns = {'date': 'datetime'})
cc_trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294588 entries, 0 to 294587
Data columns (total 5 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   credit_card                294588 non-null  int64  
 1   datetime                   294588 non-null  object 
 2   transaction_dollar_amount  294588 non-null  float64
 3   long                       294588 non-null  float64
 4   lat                        294588 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 11.2+ MB


### Data Cleaning

#### Data Duplicate

In [12]:
# check general duplicate
print(f"Total General Duplicate: {cc_trans_df.duplicated().sum()}")

Total General Duplicate: 0


In [13]:
# check specific duplicate
print(f"Total Specific Duplicate: {cc_trans_df['datetime'].duplicated().sum()}")

Total Specific Duplicate: 13523


In [14]:
# see duplicate data
duplicates_subset = cc_trans_df[cc_trans_df.duplicated(subset = ['datetime', 'credit_card'], keep = False)]
duplicates_subset = duplicates_subset.sort_values(by = ['datetime', 'credit_card'], ascending = True)
duplicates_subset.head(10)

,credit_card,datetime,transaction_dollar_amount,long,lat
69871,3253141560871065,2015-08-01 20:56:37,86.84,47.097400,26.228185
70172,3253141560871065,2015-08-01 20:56:37,32.62,-84.480998,39.119678
41771,2238144513466760,2015-08-01 22:16:36,31.57,-78.870381,42.847171
41933,2238144513466760,2015-08-01 22:16:36,25.10,-78.915551,42.888953
30944,1981294676906345,2015-08-07 19:49:41,50.82,-70.293991,43.587676
30961,1981294676906345,2015-08-07 19:49:41,38.61,-70.288511,43.662232
143776,5275410446848007,2015-08-13 19:52:24,68.29,-80.141105,40.252610
143933,5275410446848007,2015-08-13 19:52:24,83.20,-80.210411,40.279128
263107,8955392958618753,2015-08-21 16:32:58,92.85,-72.169272,43.169368
263346,8955392958618753,2015-08-21 16:32:58,56.04,81.594738,15.327278


#### Null Checking

In [15]:
# check null
cc_trans_df.isnull().sum()

credit_card                  0
datetime                     0
transaction_dollar_amount    0
long                         0
lat                          0
dtype: int64

# Combine Dataset

In [35]:
cc_df = cc_trans_df.merge(right = cc_info_df, how = 'inner', on = 'credit_card')
cc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294588 entries, 0 to 294587
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   credit_card                294588 non-null  int64         
 1   datetime                   294588 non-null  datetime64[ns]
 2   transaction_dollar_amount  294588 non-null  float64       
 3   long                       294588 non-null  float64       
 4   lat                        294588 non-null  float64       
 5   date                       294588 non-null  datetime64[ns]
 6   time                       294588 non-null  object        
 7   year                       294588 non-null  int32         
 8   month                      294588 non-null  int32         
 9   first_day_month            294588 non-null  datetime64[ns]
 10  quarter                    294588 non-null  object        
 11  day_cat                    294588 non-null  int32   

In [36]:
cc_df.head()

,credit_card,datetime,transaction_dollar_amount,long,lat,date,time,year,month,first_day_month,quarter,day_cat,day,week_cat,transaction_count,city,state,zipcode,credit_card_limit
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370,2015-09-11,00:32:40,2015,9,2015-09-01,2015Q3,4,friday,weekday,7,Houston,PA,15342,20000
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114,2015-10-24,22:23:08,2015,10,2015-10-01,2015Q4,5,saturday,weekend,7,Houston,PA,15342,20000
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004,2015-10-26,18:19:36,2015,10,2015-10-01,2015Q4,0,monday,weekday,3,Houston,PA,15342,20000
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895,2015-10-22,19:41:10,2015,10,2015-10-01,2015Q4,3,thursday,weekday,3,Houston,PA,15342,20000
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719,2015-10-26,20:08:22,2015,10,2015-10-01,2015Q4,0,monday,weekday,3,Houston,PA,15342,20000


## Data Manipulation

In [16]:
# Change Data Type
cc_trans_df['datetime'] = pd.to_datetime(cc_trans_df['datetime'], format = 'mixed', dayfirst = True)
cc_trans_df['date'] = pd.to_datetime(cc_trans_df['datetime'].dt.date, format = 'mixed', dayfirst = True)
cc_trans_df['time'] = cc_trans_df['datetime'].dt.strftime('%H:%M:%S')

# add timestamp derivation
cc_trans_df['year'] = cc_trans_df['date'].dt.year
cc_trans_df['month'] = cc_trans_df['date'].dt.month

cc_trans_df['first_day_month'] = pd.to_datetime(cc_trans_df[['year', 'month']].assign(DAY = 1))

cc_trans_df['quarter'] = pd.PeriodIndex(cc_trans_df['first_day_month'], freq = 'Q')
cc_trans_df['quarter'] = cc_trans_df['quarter'].astype(str)

cc_trans_df['day_cat'] = cc_trans_df['date'].dt.day_of_week

day_mapping = {0: 'monday', 1: 'tuesday', 2: 'wednesday', 3: 'thursday', 4: 'friday', 5: 'saturday', 6: 'sunday'}
cc_trans_df['day'] = cc_trans_df['day_cat'].map(day_mapping)

weekday_list = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday']
cc_trans_df.loc[cc_trans_df['day'].isin(weekday_list), 'week_cat'] = 'weekday'
cc_trans_df.loc[~cc_trans_df['day'].isin(weekday_list), 'week_cat'] = 'weekend'

cc_trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294588 entries, 0 to 294587
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   credit_card                294588 non-null  int64         
 1   datetime                   294588 non-null  datetime64[ns]
 2   transaction_dollar_amount  294588 non-null  float64       
 3   long                       294588 non-null  float64       
 4   lat                        294588 non-null  float64       
 5   date                       294588 non-null  datetime64[ns]
 6   time                       294588 non-null  object        
 7   year                       294588 non-null  int32         
 8   month                      294588 non-null  int32         
 9   first_day_month            294588 non-null  datetime64[ns]
 10  quarter                    294588 non-null  object        
 11  day_cat                    294588 non-null  int32   

In [17]:
cc_trans_df.head()

,credit_card,datetime,transaction_dollar_amount,long,lat,date,time,year,month,first_day_month,quarter,day_cat,day,week_cat
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370,2015-09-11,00:32:40,2015,9,2015-09-01,2015Q3,4,friday,weekday
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114,2015-10-24,22:23:08,2015,10,2015-10-01,2015Q4,5,saturday,weekend
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004,2015-10-26,18:19:36,2015,10,2015-10-01,2015Q4,0,monday,weekday
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895,2015-10-22,19:41:10,2015,10,2015-10-01,2015Q4,3,thursday,weekday
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719,2015-10-26,20:08:22,2015,10,2015-10-01,2015Q4,0,monday,weekday


In [ ]:
# Menambahkan kolom penghitung transaksi berdasarkan credit_card dan transaction_dollar_amount
duplicates_subset['transaction_count'] = duplicates_subset.groupby(['credit_card', 'datetime'])['datetime'].transform('count')
duplicates_subset.head(10)

,credit_card,datetime,transaction_dollar_amount,long,lat,transaction_count
69871,3253141560871065,2015-08-01 20:56:37,86.84,47.097400,26.228185,2
70172,3253141560871065,2015-08-01 20:56:37,32.62,-84.480998,39.119678,2
41771,2238144513466760,2015-08-01 22:16:36,31.57,-78.870381,42.847171,2
41933,2238144513466760,2015-08-01 22:16:36,25.10,-78.915551,42.888953,2
30944,1981294676906345,2015-08-07 19:49:41,50.82,-70.293991,43.587676,2
30961,1981294676906345,2015-08-07 19:49:41,38.61,-70.288511,43.662232,2
143776,5275410446848007,2015-08-13 19:52:24,68.29,-80.141105,40.252610,2
143933,5275410446848007,2015-08-13 19:52:24,83.20,-80.210411,40.279128,2
263107,8955392958618753,2015-08-21 16:32:58,92.85,-72.169272,43.169368,2
263346,8955392958618753,2015-08-21 16:32:58,56.04,81.594738,15.327278,2


In [24]:
suspicious_df = duplicates_subset.query('transaction_count > 1')
suspicious_df = suspicious_df.reset_index(drop = True)

print(f'Total suspicious rows: {len(suspicious_df)}')
suspicious_df.tail()

Total suspicious rows: 34


,credit_card,datetime,transaction_dollar_amount,long,lat,transaction_count
29,7338934618553557,2015-10-18 17:27:18,42.96,-67.064944,18.459796,2
30,2246920751887814,2015-10-21 18:15:59,49.46,-81.373006,38.148168,2
31,2246920751887814,2015-10-21 18:15:59,59.42,-81.514148,38.212790,2
32,7053196367895112,2015-10-28 21:34:58,107.37,-73.947176,40.796738,2
33,7053196367895112,2015-10-28 21:34:58,101.92,-73.924052,40.722311,2


In [30]:
# 
suspicious_df['datetime'] = pd.to_datetime(suspicious_df['datetime'], format = 'mixed', dayfirst = True)

# based on transactions durations
suspicious_df['time_diff'] = suspicious_df.groupby('credit_card')['datetime'].diff().dt.total_seconds()
suspicious_df['time_diff'] = suspicious_df['time_diff'].fillna(0)
suspicious_df.head()

,credit_card,datetime,transaction_dollar_amount,long,lat,transaction_count,time_diff
0,3253141560871065,2015-08-01 20:56:37,86.84,47.097400,26.228185,2,0.0
1,3253141560871065,2015-08-01 20:56:37,32.62,-84.480998,39.119678,2,0.0
2,2238144513466760,2015-08-01 22:16:36,31.57,-78.870381,42.847171,2,0.0
3,2238144513466760,2015-08-01 22:16:36,25.10,-78.915551,42.888953,2,0.0
4,1981294676906345,2015-08-07 19:49:41,50.82,-70.293991,43.587676,2,0.0


In [33]:
# based on geolocation
from geopy.distance import geodesic

def calculate_distance(row):
    if pd.isnull(row['prev_long']) or pd.isnull(row['prev_lat']):
        return 0
    current = (row['lat'], row['long'])
    previous = (row['prev_lat'], row['prev_long'])
    return geodesic(current, previous).kilometers

suspicious_df['prev_long'] = suspicious_df.groupby('credit_card')['long'].shift(1)
suspicious_df['prev_long'] = suspicious_df['prev_long'].fillna(0)

suspicious_df['prev_lat'] = suspicious_df.groupby('credit_card')['lat'].shift(1)
suspicious_df['prev_lat'] = suspicious_df['prev_lat'].fillna(0)

suspicious_df['distance'] = suspicious_df.apply(calculate_distance, axis=1)
suspicious_df['distance'] = suspicious_df['distance'].fillna(0)

suspicious_df.head()

,credit_card,datetime,transaction_dollar_amount,long,lat,transaction_count,time_diff,prev_long,prev_lat,distance
0,3253141560871065,2015-08-01 20:56:37,86.84,47.097400,26.228185,2,0.0,NaN,NaN,0.000000
1,3253141560871065,2015-08-01 20:56:37,32.62,-84.480998,39.119678,2,0.0,47.097400,26.228185,11200.932054
2,2238144513466760,2015-08-01 22:16:36,31.57,-78.870381,42.847171,2,0.0,NaN,NaN,0.000000
3,2238144513466760,2015-08-01 22:16:36,25.10,-78.915551,42.888953,2,0.0,-78.870381,42.847171,5.930321
4,1981294676906345,2015-08-07 19:49:41,50.82,-70.293991,43.587676,2,0.0,NaN,NaN,0.000000


In [34]:
anomaly_geo_transactions = suspicious_df[(suspicious_df['distance'] > 500) & (suspicious_df['time_diff'] < 3600)]
anomaly_geo_transactions

,credit_card,datetime,transaction_dollar_amount,long,lat,transaction_count,time_diff,prev_long,prev_lat,distance
1,3253141560871065,2015-08-01 20:56:37,32.62,-84.480998,39.119678,2,0.0,47.097400,26.228185,11200.932054
9,8955392958618753,2015-08-21 16:32:58,56.04,81.594738,15.327278,2,0.0,-72.169272,43.169368,12998.994072


# Another

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(data['transaction_dollar_amount'], bins=50, kde=True)
plt.title('Distribusi Jumlah Transaksi')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Long', y='Lat', hue='credit_card', data=data, palette='tab10')
plt.title('Distribusi Lokasi Transaksi')
plt.show()
